In [1]:
!pip install z3-solver

     --------------------------------------- 57.9/57.9 MB 16.8 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\Matteo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
from z3 import *
import itertools, math

In [3]:
instance = {
  "m" : 3,
  "n" : 7,
  "D" : [[0, 3, 3, 6, 5, 6, 6, 2],
      [3, 0, 4, 3, 4, 7, 7, 3],
      [3, 4, 0, 7, 6, 3, 5, 3],
      [6, 3, 7, 0, 3, 6, 6, 4],
      [5, 4, 6, 3, 0, 3, 3, 3],
      [6, 7, 3, 6, 3, 0, 2, 4],
      [6, 7, 5, 6, 3, 2, 0, 4],
      [2, 3, 3, 4, 3, 4, 4, 0]],
  "l" : [15, 10, 7],
  "s" : [3, 2, 6, 8, 5, 4, 4]
}

In [4]:
def array_max(vs):
  m = vs[0]
  for v in vs[1:]:
    m = If(v > m, v, m)
  return m

In [5]:
# Soluzione alternativa per l'accesso agli array che dovrebbe essere più veloce ma non va
# Test
#
#  size_vector = Array("size_vector", IntSort(), IntSort())
#  distance_matrix = Array("distance_matrix", IntSort(), IntSort())
#  for i in range(n):
#    size_vector = Store(size_vector, i, s[i])
#
#  for x, y in itertools.product(range(n + 1), range(n + 1)):
#    index = x + y * n
#    distance_matrix = Store(distance_matrix, index, D[x][y])
    #distances.append(Sum(
    #    [distance_matrix[c[i][j] - 1  + (c[i][j + 1] - 1) * n] for j in range(0,n)]
    #    ))

    #weights.append(Sum([size_vector[c[i][j]] for j in range(1,n)]))

In [6]:
def multiple_couriers(m, n, D, l, s, min_dist):

  # i is the courier
  # j is the step
  c = [[Int(f"c_{i}_{j}")  for j in range(n+2)] for i in range(m)]

  # Array indicating how much weight the courier i transport
  weights = []

  #Array indicating how much distance the courier i travel
  distances = []

  # list of coordinates of the matrix
  coords = list(itertools.product(range(m), range(n+2)))

  solver = Solver()

  for i in range(m):
    # Since we should sum D[c[i,j], c[i,j+1]] but we can't use the array c as an index we have do the If to check every possible
    # combination of c[i,j], c[i,j+1]. The range of value for k and w is [1, n+1] since we have to include
    # the value for the base n+1
    distances.append(Sum(
          [
              If(And(c[i][j] == k, c[i][j+1] == w), D[k - 1][w - 1], 0)
              for j in range(0, n) for k in range(1, n+2) for w in range(1,n+2)
          ]
    ))

    # We have the same problem for the index
    weights.append(Sum(
            [
                If(c[i][j] == k, s[k-1], 0)
                for j in range (1,n) for k in range(1, n+1)
            ])
      )

  # Constraint

  # Value range of decision variable
  for x,y in coords:
    solver.add(c[x][y] >= 1)
    solver.add(c[x][y] <= n+1)


  # A package is carried by only one courier only one time
  for x,y in coords:
    solver.add([Or(c[x][y]== n+1, c[x][y] != c[i][j]) for i,j in coords if (x,y) != (i,j)])

  # Each package is assigned to a courier
  for package in range(1, n+1):
    solver.add(Or([c[i][j] == package for i,j in coords]))


  # The total weight carried by each courier is less than or equal to his maximum carriable weight
  for i in range(m):
    solver.add(weights[i] <= l[i])

  # Every carried package must be delivered to destination and every courier must start from destination
  for i in range(m):
    solver.add(And(c[i][0] == n+1, c[i][n+1] == n+1))

  # Couriers must immediately start with a package after the base
  for i in range(m):
    for j in range(1,n+1):
      solver.add(Implies(c[i][j] != n+1 , c[i][1]!= n+1))

  # Couriers cannot go back to the base before taking other packages
  for i in range(m):
    for j in range(1,n):
      for k in range(j,n+1):
        solver.add(Implies(c[i][j] == n+1, c[i][k]==n+1))

  # Symmetry breaking
  # TODO



  # Optimization

  # Getting maximum distance
  max_distance = array_max(distances)

  # The max distance must be less than the minimum distance
  solver.add(max_distance < min_dist)


  sol = solver.check()
  if sol == sat: #If it's sat, we want to print the assignment
    g = solver.model()
    print(f"Max distance: {g.eval(max_distance)}")
    print(g.eval(distances[0]))
    print(g.eval(distances[1]))
    print(g.eval(distances[2]))
    print(g.eval(weights[0]))


    #print(solver.check())
    #for i in range(m):
    #   print()
    #   print(f"Courier {i+1}:")
    #   for j in range(n+2):
    #      print(f"Time {j}: {g[c[i][j]]}")
    return g
  else:
    print("Not sat")
    return False

In [7]:
def multiple_couriers2(m, n, D, l, s, min_dist):
    solver = Solver()

    # So that the package representing the base doens't count in the weight calculation
    s += [0]

    # Ranges
    package_range = range(n+1)
    time_range = range(n+2)
    time_range_no_zero = range(1, time_range[-1])
    courier_range = range(m)

    # Constant
    base_package = n
    last_time = n + 1
    coords = list(itertools.product(courier_range, time_range))


    # Variables
    # y[p][t][c] == 1 se c porta p al tempo t
    y = [[[Int( f"y_{package}_{time}_{courier}")
          for courier in courier_range]
          for time in time_range]
          for package in package_range]

    # Binary constraint
    # Value range of decision variable
    for package in package_range:
      for courier in courier_range:
        for time in time_range:
          solver.add(y[package][time][courier] <= 1)
          solver.add(y[package][time][courier] >= 0)


    # weights vector, weights[c] is the amount transported by c
    weights = []
    for courier in courier_range:
      weights.append(
          Sum([s[package] * y[package][time][courier] for time in time_range for package in package_range])
      )

    # distances vector, distances[c] is the amount travelled by c
    distances = []
    for courier in courier_range:
      courier_distances = []
      for time in time_range_no_zero:
        for p1 in package_range:
          for p2 in package_range:
            a = y[p1][time - 1][courier] == 1
            b = y[p2][time][courier] == 1
            courier_distances.append(D[p1][p2] * And(a, b))

      distances.append(Sum(courier_distances))


    # Each package taken one time only
    for package in package_range:
      if package == base_package:
        continue
      solver.add(Sum([y[package][time][courier] for time in time_range for courier in courier_range]) == 1)

    # A courier carry only one package at time
    for courier in courier_range:
      for time in time_range:
        solver.add(Sum([y[package][time][courier] for package in package_range]) == 1)


    # Every carried package must be delivered to destination and every courier must start from destination
    for courier in courier_range:
        solver.add(y[base_package][0][courier] == 1)
        solver.add(y[base_package][last_time][courier] == 1)

    # Each courier can hold packages with total weight <= max carriable weight
    for courier in courier_range:
        solver.add(weights[courier] <= l[courier])

    #Couriers must immediately start with a package after the base
    for courier in courier_range:
      for time in time_range_no_zero:
        a = y[base_package][time][courier]
        b = y[base_package][1][courier]

        solver.add(Implies(a != 1, b != 1))

    #Couriers cannot go back to the base before taking other packages
    for courier in courier_range:
      for time in time_range_no_zero:
        a = y[base_package][time][courier]

        for time2 in range(time + 1, last_time):
          b = y[base_package][time2][courier]
          solver.add(Implies(a == 1, b == 1))

    # Heuristic (?)
    max_package = math.ceil(n / m)
    #print(max_package)
    for courier in courier_range:
      package_transported = Sum([
          y[package][time][courier] for time in time_range for package in package_range if package != base_package
      ])


      solver.add(package_transported <= max_package)



    # Getting maximum distance
    max_distance = array_max(distances)

    # The max distance must be less than the minimum distance
    solver.add(max_distance <= min_dist)


    sol = solver.check()
    if sol == sat: #If it's sat, we want to print the assignment
      g = solver.model()
      print(f"Max distance: {g.eval(max_distance)}")
      #print(g.eval(y))
      #print(g.eval(distances[0]))
      #print(g.eval(distances[1]))
      #print(g.eval(distances[2]))
      #print(g.eval(weights[0]))


      #print(solver.check())

      for courier in courier_range:
        print("\n")
        t = ""
        for time in time_range:
          value = sum(package * g.eval(y[package][time][courier]) for package in package_range)
          t += f"{g.eval(value + 1)}, "

        print(t)

      return g
    else:
      print("Not sat")
      return False

In [8]:
from time import time

MAXITER = 500

def minimizer_binary(instance, solver = multiple_couriers, maxiter = MAXITER):
  m = instance["m"]
  n = instance["n"]
  D = instance["D"]
  l = instance["l"]
  s = instance["s"]
  start = time()

  min_dist = 9999999
  for i in range(len(D)):
    for j in range(len(D[i])):
      if D[i][j] <= min_dist and D[i][j] != 0:
        min_dist = D[i][j]

  max_dist = 0
  for i in range(len(D)):
    max_dist += max(D[i])

  iter = 1
  while iter < maxiter:
    k = int((min_dist + max_dist) / 2)

    print(f"Current k={k}")

    sol = solver(m, n, D, l, s, k)
    if not sol:
      min_dist = k
    else:
      max_dist = k
    if abs(min_dist - max_dist) <= 1:
      return max_dist, sol, f"{time() - start:.2f}", iter


      #if sol:
      #  return max_dist, sol, f"{time() - start:.2f}", iter
      #else:
      #  return max_dist, "Unsat", f"{time() - start:.2f}", iter
    iter+=1
  return max_dist, "Out of time", f"{time() - start:.2f}", iter

In [9]:
min_dist, sol, time_passed, iter = minimizer_binary(instance, multiple_couriers2)
print("\nResults:")
if type(sol) == str:
  print(f"Solution: {sol}")
print(f"Minimum distance found is {min_dist}")
print(f"Time: {time_passed}")
print(f"Iterations: {iter}")

Current k=26
Max distance: 20


8, 4, 1, 7, 8, 8, 8, 8, 8, 


8, 3, 6, 8, 8, 8, 8, 8, 8, 


8, 2, 5, 8, 8, 8, 8, 8, 8, 
Current k=14
Max distance: 12


8, 5, 6, 3, 8, 8, 8, 8, 8, 


8, 4, 2, 8, 8, 8, 8, 8, 8, 


8, 7, 1, 8, 8, 8, 8, 8, 8, 
Current k=8
Not sat
Current k=11
Not sat
Current k=12
Max distance: 12


8, 2, 4, 5, 8, 8, 8, 8, 8, 


8, 6, 3, 8, 8, 8, 8, 8, 8, 


8, 1, 7, 8, 8, 8, 8, 8, 8, 

Results:
Minimum distance found is 12
Time: 2.37
Iterations: 5
